In [ ]:
SAM 2 (Segment Anything Model 2)

sam2_capabilities = {
    "тип": "Zero-shot сегментация",
    "архитектура": "ViT-H + prompt encoder",
    "сильные_стороны": [
        "🎯 Сегментация любых объектов без обучения",
        "🚀 Работает по точкам, bbox, маскам",
        "🔍 Высокое качество границ",
        "📊 Поддержка видео"
    ],
    "использование": [
        "Интерактивная сегментация",
        "Создание масок для обучения",
        "Медицинская сегментация",
        "Удаление фона"
    ]
}